# 0） Gererate the Data and Preparation

In [ ]:
import numpy as np
import cvxpy as cp
import random
import pandas as pd

-- Load from csv

In [ ]:
n = 10000    # number of bidders
m = 10       # type of resources

A = pd.read_csv("A.csv")
pi = pd.read_csv("pi.csv")

A = np.array(A)
pi = np.array(pi)
b = np.array([1000]*10).reshape(-1,1)

print(f'The shape of A :  {A.shape}')
print(f'The shape of pi:  {pi.shape}')
print(f'The shape of b :  {b.shape}')
print(f'The dtype of A :  {A.dtype}')
print(f'The dtype of pi:  {pi.dtype}')
print(f'The dtype of b :  {b.dtype}')

The shape of A :  (10, 10000)
The shape of pi:  (10000, 1)
The shape of b :  (10, 1)
The dtype of A :  int64
The dtype of pi:  float64
The dtype of b :  int64


-- set the random seed for reproducibility

In [ ]:
# for reproducibility
random.seed(4300)

n = 10000    # number of bidders
m = 10       # type of resources
p = np.array([float(random.gauss(0.5,0.1)) for _ in range(m)])      # ground truth price vector: m*1

pi = (p.T).dot(A)
for i in range(n):
    pi[i] += random.gauss(0, 0.2)
    
b = np.array([1000]*10).reshape(-1,1)

print(f'The shape of A :  {A.shape}')
print(f'The shape of pi:  {pi.shape}')
print(f'The shape of b :  {b.shape}')                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

# 1) One-time Online Learning (SLPM)

## Offline Learning

In [ ]:
x = cp.Variable((n,1))
objective = cp.Maximize(pi.T@x)
constraints = [A@x <= b, x >= 0, x <= 1]
prob = cp.Problem(objective, constraints)

prob.solve()

opv1 = prob.value
x = np.where(x.value >= 0.5, 1, 0)

print(f'Optimal Solution: \n{x[:,0].T}')
print(f'Optimal Value: {opv1}')

Optimal Solution: 
[1 0 1 ... 0 0 0]
Optimal Value: 282270.0744458285


## One-time Online Learning (SLPM) Algorithm Implement and Sensitivity Analysis

In [ ]:
def online_learning_SLPM(A, pi, b, n, k):
  # solve the dual SLPM with k bidders 
  m = len(b)
  y = cp.Variable((m+k,1))
  A_prime = np.hstack([A[:,:k].T, np.eye(k)])
  b_prime = np.append((k/n)*b,np.ones(k)).reshape(1,-1)
  pi_prime = pi[:k,:]
  objective = cp.Minimize(b_prime@y)
  constraints = [A_prime@y >= pi_prime, y >= 0]
  prob = cp.Problem(objective, constraints)
  prob.solve()

  # obtain the dual prices
  P = y[:m,:].value
  
  # proposal of x
  x = np.where(pi>A.T@P,1,0)
  x[:k,:] = 0
  # output x
  sold = np.zeros_like(b)
  for i in range(k,n):
    if ((A[:,i].T*x[i,:]).reshape(-1,1) > b-sold).any():
      x[i,:] = 0
    else: sold += (A[:,i].T*x[i,:]).reshape(-1,1)

  # ouptut the optimal revenue
  revenue = (pi.T@x)[0,0]
  return P, x, revenue


In [ ]:
print('-'*25 + 'One-time Online Learning SLPM' + '-'*25 + '\n')
for k in [50, 100, 200]:
  P,x,r = online_learning_SLPM(A,pi,b,10000,k)
  print('-'*25 + f'k = {k}' + '-'*25 + '\n')
  print(f'Dual price: {P.T[0,:]} \n')
  print(f'Allocation: {x.T[0,:]} \n')
  print(f'Revenue: {r} \n')
  print(f'Ratio: {r/opv1} \n')

-------------------------One-time Online Learning SLPM-------------------------

-------------------------k = 50-------------------------

Dual price: [44.08131118 34.73590639 14.2834981  49.16637156 25.32845307 26.23525563
  9.9796159  25.22667587 13.64073604 39.06147426] 

Allocation: [0 0 0 ... 0 0 0] 

Revenue: 202872.86352702987 

Ratio: 0.7187189925298438 

-------------------------k = 100-------------------------

Dual price: [44.10761798 35.05640017 13.94883047 49.06530233 25.1132787  26.04832173
 10.14120411 25.06083749 13.97303324 39.19661529] 

Allocation: [0 0 0 ... 0 0 0] 

Revenue: 250940.8308227826 

Ratio: 0.8890096880281994 

-------------------------k = 200-------------------------

Dual price: [44.08445376 35.01221617 14.04305541 49.0984615  24.97019652 26.0279769
 10.1003609  25.0784745  13.97691391 39.1746032 ] 

Allocation: [0 0 0 ... 0 0 0] 

Revenue: 262596.95388262294 

Ratio: 0.9303039098217225 



## Comments

As k increasing, the ratio of the generated revenue over the offline revenue increases. 

Trade-off:---------------------------------------------------------------------------

# 2) Dynamic Learning (SLPM)

## Dynamic Learning (SLPM) Algorithm Implement

In [ ]:
def dynamic_learning_SLPM(A, pi, b, n, k):
  global x_out
  x_out = np.zeros_like(pi)
  kk = k
  m = len(b)
  print('-'*30 + 'The Track of Dual Price' + '-'*30 + '\n')
  while kk < n:
    # solve the dual SLPM with kk bidders 
    y = cp.Variable((m+kk,1))
    A_prime = np.hstack([A[:,:kk].T, np.eye(kk)])
    b_prime = np.append((kk/n)*b,np.ones(kk)).reshape(1,-1)
    pi_prime = pi[:kk,:]
    objective = cp.Minimize(b_prime@y)
    constraints = [A_prime@y >= pi_prime, y >= 0]
    prob = cp.Problem(objective, constraints)
    prob.solve()

    # obtain the dual prices
    P = y[:m,:].value
    print('-'*25 + f'k = {kk}', '-'*25 + '\n')
    print(f'Dual price: {P.T[0,:]}\n')

    # proposal of x
    x = np.where(pi>A.T@P,1,0)
    x[:kk,:] = x_out[:kk,:]

    # update x
    sold = A[:,:kk]@x_out[:kk,:]
    for i in range(kk,min(2*kk, n)):
      if ((A[:,i].T*x[i,:]).reshape(-1,1) > b-sold).any():
        x[i,:] = 0
      else: sold += (A[:,i].T*x[i,:]).reshape(-1,1)
    
    # update x_out
    x_out[kk:min(2*kk, n),:] = x[kk:min(2*kk, n),:]

    # update kk
    kk += kk
  
  # ouptut the optimal revenue
  revenue = (pi.T@x_out)[0,0]
  print('-'*40 + 'END' + '-'*40)
  return P, x_out, revenue

In [ ]:
P,x,r = dynamic_learning_SLPM(A, pi, b, 10000, 50)

------------------------------The Track of Dual Price------------------------------

-------------------------k = 50 -------------------------

Dual price: [44.08131118 34.73590639 14.2834981  49.16637156 25.32845307 26.23525563
  9.9796159  25.22667587 13.64073604 39.06147426]

-------------------------k = 100 -------------------------

Dual price: [44.10761798 35.05640017 13.94883047 49.06530233 25.1132787  26.04832173
 10.14120411 25.06083749 13.97303324 39.19661529]

-------------------------k = 200 -------------------------

Dual price: [44.08445376 35.01221617 14.04305541 49.0984615  24.97019652 26.0279769
 10.1003609  25.0784745  13.97691391 39.1746032 ]

-------------------------k = 400 -------------------------

Dual price: [44.11575924 35.02564286 14.0975596  49.11769075 25.0724957  26.02435099
 10.13486356 25.08632994 13.91276816 39.10288637]

-------------------------k = 800 -------------------------

Dual price: [44.08627972 35.09342037 14.07815968 49.07971733 25.07023365 

In [ ]:
# groud truth dual price
y = cp.Variable((m+n,1))
A_prime = np.hstack([A.T, np.eye(n)])
b_prime = np.append(b,np.ones(n)).reshape(1,-1)
pi_prime = pi
objective = cp.Minimize(b_prime@y)
constraints = [A_prime@y >= pi_prime, y >= 0]
prob = cp.Problem(objective, constraints)
prob.solve()
print(f'The optimal revenue: {prob.value}')
print(f'The ground truth dual price: \n{y.value[:10,:].T[0,:]}')

The optimal revenue: 282270.07444582676
The ground truth dual price: 
[44.10011278 35.06752221 14.06044963 49.07278232 25.07418756 26.07114384
 10.0602146  25.07050949 14.0626997  39.05658465]


In [ ]:
print('-'*30 + 'Dynamic Learning SCLM' + '-'*30 + '\n')
print(f'Dual price: {P.T[0,:]} \n')
print(f'Allocation: {x.T[0,:]} \n')
print(f'Revenue: {r} \n')
print(f'Ratio: {r/opv1} \n')

------------------------------Dynamic Learning SCLM------------------------------

Dual price: [44.1011333  35.06522694 14.05213781 49.058396   25.07480728 26.08178129
 10.05461491 25.08021312 14.06390897 39.06170895] 

Allocation: [0. 0. 0. ... 0. 0. 0.] 

Revenue: 275042.4146179024 

Ratio: 0.9743945232518328 



## Comments

--------------------------------------------------

# 3) Proof

# 4) One-time Online Learning (SCPM)

## One-time Online Learning (SCPM) Algorithm Implement

In [ ]:
!pip install SCS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def online_learning_SCPM(A, pi, b, n, k, w):
  # solve the primal SCPM with k bidders 
  m = len(b)
  b = b.reshape(m)
  y = cp.Variable(k)
  s = cp.Variable(m)
  objective = cp.Maximize(pi[:k,:].T@y + (w/m)*cp.sum(cp.log(s)))
  constraints = [A[:,:k]@y + s <= (k/n)*b, y >= 0, y <= 1]
  prob = cp.Problem(objective, constraints)
  prob.solve(cp.SCS)

  # obtain the dual prices
  P = prob.constraints[0].dual_value.reshape(-1,1)
  
  # proposal of x
  x = np.where(pi>A.T@P,1,0)
  x[:k,:] = 0

  # output x
  b = b.reshape(-1,1)
  sold = np.zeros_like(b)
  for i in range(k,n):
    if ((A[:,i].T*x[i,:]).reshape(-1,1) > b-sold).any():
      x[i,:] = 0
    else: sold += (A[:,i].T*x[i,:]).reshape(-1,1)

  # ouptut the optimal revenue
  revenue = (pi.T@x)[0,0]
  return P, x, revenue


In [ ]:
print('-'*25 + 'One-time Online Learning SCPM' + '-'*25 + '\n')
for k in [50, 100, 200]:
  P,x,r = online_learning_SCPM(A,pi,b,10000,k,1)
  print('-'*25 + f'k = {k}' + '-'*25 + '\n')
  print(f'Dual price: {P.T[0,:]} \n')
  print(f'Allocation: {x.T[0,:]} \n')
  print(f'Revenue: {r} \n')
  print(f'Ratio: {r/opv1} \n')

-------------------------One-time Online Learning SCPM-------------------------

-------------------------k = 50-------------------------

Dual price: [44.08107692 34.73455253 14.28391643 49.16567586 25.3306513  26.23536665
  9.9793468  25.22723772 13.64142672 39.05960151] 

Allocation: [0 0 0 ... 0 0 0] 

Revenue: 202844.59559326863 

Ratio: 0.7186188475399198 

-------------------------k = 100-------------------------

Dual price: [44.0952587  35.03066015 13.98477837 49.09057023 25.0396567  26.01271965
 10.18851328 25.08728669 13.9836784  39.14983997] 

Allocation: [0 0 0 ... 0 0 0] 

Revenue: 259779.69410064444 

Ratio: 0.9203231855543359 

-------------------------k = 200-------------------------

Dual price: [44.08538016 35.00755035 14.04530684 49.09774847 24.96418302 26.03348051
 10.09854051 25.08253542 13.98054605 39.17291032] 

Allocation: [0 0 0 ... 0 0 0] 

Revenue: 261796.93642631202 

Ratio: 0.9274696828570627 



## Comments

# 5) Dynamic Learning (SCPM)

## Dynamic Learning (SCPM) Algorithm Implement

In [ ]:
def dynamic_learning_SCPM(A, pi, b, n, k, w):
  global x_out
  x_out = np.zeros_like(pi)
  kk = k
  m = len(b)
  print('-'*30 + 'The Track of Dual Price' + '-'*30 + '\n')
  while kk < n:
    # solve the primal SCPM with k bidders 
    b = b.reshape(m)
    y = cp.Variable(kk)
    s = cp.Variable(m)
    objective = cp.Maximize(pi[:kk,:].T@y + (w/m)*cp.sum(cp.log(s)))
    constraints = [A[:,:kk]@y + s <= (kk/n)*b, y >= 0, y <= 1]
    prob = cp.Problem(objective, constraints)
    prob.solve(cp.SCS)

    # obtain the dual prices
    P = prob.constraints[0].dual_value.reshape(-1,1)
    print('-'*25 + f'k = {kk}', '-'*25 + '\n')
    print(f'Dual price: {P.T[0,:]}\n')

    # proposal of x
    x = np.where(pi>A.T@P,1,0)
    x[:kk,:] = x_out[:kk,:]
    
    # update x
    b = b.reshape(-1,1)
    sold = A[:,:kk]@x_out[:kk,:]
    for i in range(kk,min(2*kk, n)):
      if ((A[:,i].T*x[i,:]).reshape(-1,1) > b-sold).any():
        x[i,:] = 0
      else: sold += (A[:,i].T*x[i,:]).reshape(-1,1)
    
    # update x_out
    x_out[kk:min(2*kk, n),:] = x[kk:min(2*kk, n),:]

    # update kk
    kk += kk
  
  # ouptut the optimal revenue
  revenue = (pi.T@x_out)[0,0]
  print('-'*40 + 'END' + '-'*40)
  return P, x_out, revenue

In [ ]:
P,x,r = dynamic_learning_SCPM(A, pi, b, 10000, 50, 1)

------------------------------The Track of Dual Price------------------------------

-------------------------k = 50 -------------------------

Dual price: [44.08107692 34.73455253 14.28391643 49.16567586 25.3306513  26.23536665
  9.9793468  25.22723772 13.64142672 39.05960151]

-------------------------k = 100 -------------------------

Dual price: [44.0952587  35.03066015 13.98477837 49.09057023 25.0396567  26.01271965
 10.18851328 25.08728669 13.9836784  39.14983997]

-------------------------k = 200 -------------------------

Dual price: [44.08538016 35.00755035 14.04530684 49.09774847 24.96418302 26.03348051
 10.09854051 25.08253542 13.98054605 39.17291032]

-------------------------k = 400 -------------------------

Dual price: [44.11565667 35.02517968 14.09768505 49.11828489 25.07227907 26.02423411
 10.13504165 25.08657958 13.91264146 39.10267076]

-------------------------k = 800 -------------------------

Dual price: [44.08617397 35.09295865 14.07896307 49.08009142 25.07003973

In [ ]:
# groud truth dual price
w = 1
b = b.reshape(m)
y = cp.Variable(n)
s = cp.Variable(m)
objective = cp.Maximize(pi[:,:].T@y + (w/m)*cp.sum(cp.log(s)))
constraints = [A@y + s <= b, y >= 0, y <= 1]
prob = cp.Problem(objective, constraints)
prob.solve(cp.SCS)
b = b.reshape(-1,1)
print(f'The optimal revenue: {prob.value}')
print(f'The ground truth dual price: \n{prob.constraints[0].dual_value}')

The optimal revenue: 282263.54696186224
The ground truth dual price: 
[44.10029288 35.06748989 14.06033983 49.07277549 25.07413983 26.07132079
 10.06029218 25.07008807 14.06288852 39.05657544]


In [ ]:
print('-'*30 + 'Dynamic Learning SCPM' + '-'*30 + '\n')
print(f'Dual price: {P.T[0,:]} \n')
print(f'Allocation: {x.T[0,:]} \n')
print(f'Revenue: {r} \n')
print(f'Ratio: {r/opv1} \n')

------------------------------Dynamic Learning SCPM------------------------------

Dual price: [44.10121057 35.06491512 14.05197554 49.05847329 25.07503806 26.08154817
 10.05433993 25.08045335 14.0638946  39.06186425] 

Allocation: [0. 0. 0. ... 0. 0. 0.] 

Revenue: 275449.66181122075 

Ratio: 0.9758372804910402 



## Comments

--------------------------------------

# 6) Action-history-dependent Learning Algorithm

In [ ]:
A

array([[1, 0, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 1, 1, 1],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 1, 0],
       [0, 1, 1, ..., 1, 1, 1]])

In [ ]:
A[:,0]

array([1, 0, 1, 1, 0, 1, 1, 1, 0, 0])

In [ ]:
def aciton_history_dependent_learning(A, pi, b, n):
  global bb
  bb = b 
  x = np.zeros_like(pi)
  for k in range(1,n):
    # solve the dual SLPM with k bidders 
    m = len(bb)
    y = cp.Variable((m+k,1))
    A_prime = np.hstack([A[:,:k].T, np.eye(k)])
    b_prime = np.append((k/(n-k))*bb,np.ones(k)).reshape(1,-1)
    pi_prime = pi[:k,:]
    objective = cp.Minimize(b_prime@y)
    constraints = [A_prime@y >= pi_prime, y >= 0]
    prob = cp.Problem(objective, constraints)
    prob.solve()

    # obtain the dual prices
    P = y[:m,:].value

    # determine the value of x
    x[k-1,:] = 1 if pi[k-1,0] > (A[:,k-1]@P)[0] and (A[:,k-1].reshape(-1,1) <= bb).all() else 0

    # update bb
    if x[k-1,:] == 1:
      bb = bb - A[:,k-1].reshape(-1,1)

    # keep track of the program
    if k%1000 == 0:
      print('-'*30 + f'k = {k}' + '-'*30)
      print(f'current remaining goods: {bb.T}')
      print(f'current dual price: {P.T}')
      print(f'current allocated number: {int(np.sum(x))}')

  # ouptut the optimal revenue
  revenue = (pi.T@x)[0,0]
  return P, x, revenue


In [ ]:
P,x,r = aciton_history_dependent_learning(A, pi, b, 10000)

------------------------------k = 1000------------------------------
current remaining goods: [[896 897 895 906 899 890 903 900 895 903]]
current dual price: [[44.08756915 35.0762377  14.10087913 49.05854786 25.08382948 26.06760534
  10.11527178 25.05851014 13.94503273 39.13470316]]
current allocated number: 222
------------------------------k = 2000------------------------------
current remaining goods: [[793 808 810 814 806 800 816 802 786 815]]
current dual price: [[44.11379581 35.04508342 14.06041217 49.04932309 25.07534474 26.07364114
  10.0878957  25.08125644 14.05825098 39.04690237]]
current allocated number: 435
------------------------------k = 3000------------------------------
current remaining goods: [[699 708 717 712 712 707 724 697 689 717]]
current dual price: [[44.11584939 35.04589005 14.04564521 49.06458464 25.06758621 26.05150952
  10.06077005 25.11780291 14.06786791 39.04504396]]
current allocated number: 643
------------------------------k = 4000--------------------

KeyboardInterrupt: ignored

In [ ]:
print(f'Dual price: {P.T[0,:]} \n')
print(f'Allocation: {x.T[0,:]} \n')
print(f'Revenue: {r} \n')
print(f'Ratio: {r/opv1} \n')